# DBMI SET Effort Analysis - Data Exploration - Toggl.com

In [ ]:
import os
import pathlib
import zipfile
from datetime import datetime, timedelta

import duckdb
import pandas as pd
from toggl.TogglPy import Toggl

In [ ]:
# pull data from dvc remote
!dvc pull

In [ ]:
# setting some paths for later reference
toggl_data_dir = "../data/toggl.com/"
toggl_dir_path = pathlib.Path(toggl_data_dir).mkdir(exist_ok=True)

In [ ]:
# gather the api token from environment
api_token = os.environ["TOGGL_API_TOKEN"]

# form the toggl client
toggl = Toggl()
toggl.setAPIKey(api_token)

In [ ]:
def get_flattened_summary_data(toggl: Toggl, date: str) -> dict:
    """
    Find and return data structure which includes flattened
    summary stats from toggl workday based on date str provided
    """

    # gather a workspace for making report requests
    workspaces = toggl.getWorkspaces()
    default_workspace_id = workspaces[0]["id"]

    # specify that we want reports from this week
    req_data = {
        "workspace_id": default_workspace_id,
        "since": date,
        "until": date,
    }

    # gather the report data
    report_data = toggl.getSummaryReport(req_data)

    return report_data

In [ ]:
# gather the last 3 days of work as flattened report data
for num in range(1, 4):
    date = (datetime.now() - timedelta(num)).strftime("%Y-%m-%d")
    print(date)
    data = get_flattened_summary_data(toggl, date)

In [ ]:
data
pd.json_normalize(data, record_path=["data"]).explode("items")

In [ ]:
# export for potential later use
acct_project_tasks.to_parquet(
    "../data/monday.com/datadbmi-set-monday-acct-project-tasks.parquet"
)

In [ ]:
!dvc add ../data/monday.com/datadbmi-set-monday-acct-project-tasks.parquet
!git add ../data/monday.com/datadbmi-set-monday-acct-project-tasks.parquet.dvc
!dvc push